In [1]:
import tensorflow as tf # TODO

In [2]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [3]:
# Parameters
learning_rate = 0.1
training_epochs = 200
batch_size = 50
display_step = 5
beta = 0.1

In [4]:
def mackey_glass(t, t25):
    return 0.9 * t + 0.2*t25/(1+t25**10) + np.random.normal(0, 0.03, 1)
fdata = [1.5]
for i in range(1525):
    fdata.append(mackey_glass(fdata[i], fdata[i - 25] if i > 25 else 0))

In [5]:
def create_mg_series(i, data):
    return np.asarray([data[i-20], data[i-15], data[i-10], data[i-5], data[i]])

train_X = np.asarray(list(map(lambda x: create_mg_series(x, fdata), range(301, 800))))
train_Y = np.asarray(list(map(lambda x: fdata[x+5], range(301, 800))))
validation_X = np.asarray(list(map(lambda x: create_mg_series(x, fdata), range(801, 1300))))
validation_Y = np.asarray(list(map(lambda x: fdata[x+5], range(801, 1300))))
test_X = np.asarray(list(map(lambda x: create_mg_series(x, fdata), range(1301, 1500))))
test_Y = np.asarray(list(map(lambda x: fdata[x+5], range(1301, 1500))))

In [6]:
# X Y
X = tf.placeholder(tf.float32, [1,5], name='x')  # 5 inputs
Y = tf.placeholder(tf.float32,name='y')  # 1 output

# hidden layer 1
W1 = tf.Variable(tf.truncated_normal([5, 8], stddev=0.03), name='W1')
b1 = tf.Variable(tf.truncated_normal([8]), name='b1')

# hidden layer 2
W2 = tf.Variable(tf.truncated_normal([8, 5], stddev=0.03), name='W2')
b2 = tf.Variable(tf.truncated_normal([5]), name='b2')

In [7]:
# output hidden layer 1
hidden_out = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

# total output
y_ = tf.nn.relu(tf.add(tf.matmul(hidden_out, W2), b2))

In [8]:
mse = tf.losses.mean_squared_error(Y, y_)

In [9]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(mse)

In [10]:
# initialize variables
init_op = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    batches = int(len(train_Y) / batch_size)
    # Fit all training data
    early_stop = False
    plotlist = []
    for epoch in range(training_epochs):
        if (early_stop):
            break
        avg_cost = 0
        avg_vcost = 0
        prev_vcost = float("inf")
        test_data = list(zip(test_X, test_Y))
        validation_data = list(zip(validation_X, validation_Y))

        for i in range(len(test_data)):
            x, y = test_data[i]
            x = x.reshape(1, 5)
            y = y.reshape(1, 1)
            vx, vy = test_data[i]
            vx = vx.reshape(1, 5)
            vy = vy.reshape(1, 1)
            _, cost = sess.run([optimizer, mse], feed_dict={X: x, Y: y})
            
            avg_cost += cost / training_epochs
            vcost = sess.run(
                tf.reduce_sum(tf.pow(y_ - Y, 2)) / (2 * test_X.shape[0]),
                feed_dict={X: vx, Y: vy})  # same function as cost above
            # Early stopping
            #if (vcost > prev_vcost):
            #    print("Stop!!! {} vs {}".format(vcost, prev_vcost))
            #    early_stop = True
            #    break
                
            prev_vcost = vcost
            avg_vcost += vcost / training_epochs
            plotlist.append(cost)

        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost), "vcost=", "{:.9f}".format(avg_vcost))
        avg_cost = avg_vcost = 0
            

Epoch: 0001 cost= 0.714517563 vcost= 0.008952282
Epoch: 0002 cost= 0.712068870 vcost= 0.008927807
Epoch: 0003 cost= 0.711496371 vcost= 0.008921943
Epoch: 0004 cost= 0.711099679 vcost= 0.008917980
Epoch: 0005 cost= 0.710812853 vcost= 0.008915168
Epoch: 0006 cost= 0.710601227 vcost= 0.008913118
Epoch: 0007 cost= 0.710445706 vcost= 0.008911617
Epoch: 0008 cost= 0.710335631 vcost= 0.008910545
Epoch: 0009 cost= 0.710265219 vcost= 0.008909834
Epoch: 0010 cost= 0.710231711 vcost= 0.008909450
Epoch: 0011 cost= 0.710234048 vcost= 0.008909378
Epoch: 0012 cost= 0.710271778 vcost= 0.008909611
Epoch: 0013 cost= 0.710343771 vcost= 0.008910142
Epoch: 0014 cost= 0.710446814 vcost= 0.008910946
Epoch: 0015 cost= 0.710574160 vcost= 0.008911974
Epoch: 0016 cost= 0.710714671 vcost= 0.008913140
Epoch: 0017 cost= 0.710853313 vcost= 0.008914328
Epoch: 0018 cost= 0.710973533 vcost= 0.008915406
Epoch: 0019 cost= 0.711061299 vcost= 0.008916262
Epoch: 0020 cost= 0.711109122 vcost= 0.008916826
Epoch: 0021 cost= 0.

In [ ]:
plt.plot(plotlist)